In [1]:
import os
from detectors import *
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "C:/Users/Luiz Fernando/Projects/Quantification-in-Data-Streams/datasets"
files = os.listdir(path)
datasets = [x.split(".")[0] for x in files]
datasets = list(dict.fromkeys(datasets))
print(datasets)

['AedesQuinx', 'AedesSex', 'Arabic-digit', 'wine']


In [3]:
clf = RandomForestClassifier()
parameters = {'n_estimators':[100, 200, 300, 400, 500]}
model = GridSearchCV(estimator = clf,
                    param_grid = parameters,
                    scoring = 'accuracy',
                    cv = 10,
                    n_jobs = -1)

In [4]:
table = pd.DataFrame(index=[datasets])
window_size = 300 #window parameter to build the images for comparison
train_size = 300
table

""
AedesQuinx
AedesSex
Arabic-digit
wine


In [ ]:
for file in files:
    stream = pd.read_csv(path + "/" + file, header=0, index_col=False,sep=',')
    stream.iloc[:, -2].replace(2, int(0), inplace=True)
    train = stream.iloc[:train_size].reset_index(drop=True)
    test = stream.iloc[train_size:].reset_index(drop=True)

    print(f"Dataset: {file[:-4]}")
    for detector in ["IKS", "IBDD", "WRS", "ADWIN", "BASELINE","TOPLINE"]:
        if detector == "IKS":
            threshold = 1.95
            [drifts, mean_acc, time] = IKS(train, test, window_size, threshold, model)
        elif detector == "IBDD":
            epsilon = 3
            [drifts, acc_vector, mean_acc, time] = IBDD(train, test, window_size, epsilon, model)
        elif detector == "WRS":
            threshold = 0.001
            [drifts, acc_vector, mean_acc, time] = WRS(train, test, window_size, threshold, model)
        elif detector == "ADWIN":
            [drifts, acc_vector, mean_acc, time] = Adwin(train, test, window_size, model)
        elif detector == "TOPLINE":
            [mean_acc, acc_vector, time] = topline_classifier(train, test, window_size, model)
        elif detector == "BASELINE":
            [mean_acc, acc_vector, time] = baseline_classifier(train, test, model)
        print("=======================")
            
        table.loc[[file[:-4]], [detector]] = round(mean_acc)
table

    

Dataset: AedesQuinx.test.
IKS Running...
classle 1/5700 - drifts: []
1    300
Name: count, dtype: int64
[1.0, 0.0]
[1, 0]
[1, 0]
[1.0, 0.0]
{'CC': [1.0, 0.0], 'ACC': [1, 0], 'MS': [1, 0], 'DyS': [1.0, 0.0]}
class
1    1.0
Name: proportion, dtype: float64
   IKS  IKS-CC  IKS-ACC  IKS-MS  IKS-DyS
0  1.0     1.0      1.0     1.0      1.0
classle 2/5700 - drifts: []
1    300
Name: count, dtype: int64
[1.0, 0.0]


# INCREMENTAL KOLMOGOROV-SMIRNOV (IKS):

In [ ]:
FILENAME_TRAIN = 'benchmark_real/Yoga_TRAIN.data'
FILENAME_TEST = 'benchmark_real/Yoga_TEST.data'
threshold = 1.12
[drifts, acc_vector, mean_acc, time] = IKS(FILENAME_TRAIN, FILENAME_TEST, window_size, threshold)

# IMAGE-BASED DRIFT DETECTION (IBDD):

In [ ]:
epsilon = 3 #number of MSD values above/below threshold
[drifts, acc_vector, mean_acc, time] = IBDD(FILENAME_TRAIN, FILENAME_TEST, window_size, epsilon)

# BASELINE:

In [ ]:
[mean_acc, acc_vector, time] = baseline_classifier(FILENAME_TRAIN, FILENAME_TEST)

# TOPLINE:

In [ ]:
[mean_acc, acc_vector, time] = topline_classifier(FILENAME_TRAIN, FILENAME_TEST, window_size)

# WILCOXON-RANK SUM (WRS):

In [ ]:
threshold = 0.001
[drifts, acc_vector, mean_acc, time] = wrs_test(FILENAME_TRAIN, FILENAME_TEST, window_size, threshold)